# A Set of Analyses for Questacon

## Imports

In [ ]:
import os
import pickle
from math import floor
from itertools import repeat
from multiprocessing import Pool
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from apsrm import PCRTest, Person, Box
from apsrm.config import POOL_NCORES, DEFAULT_STRAIN
from apsrm.ext.multiprocessing import ProcessSeeder
from apsrm.ext.simulation import create_pathogen
from apsrm.ext.plot import (
    shedding_plot,
    extract_plot_data,
    plot_concentrations)
from utils import (
    BOX,
    BOX_NAME_TO_BOX_ENUM,
    HUMAN_BOX_TYPES,
    VisitGenerator,
    create_workplace,
    create_emissions_calculator,
    generate_visitors_for_period)

## Params and Functions

In [ ]:
pathogen_name = DEFAULT_STRAIN
bau_n_visitors = 1400
limited_n_visitors = 350

OUTPUT_BASE_DIR = '../../outputs/questacon'
def opath(p):
    output_base_dir = OUTPUT_BASE_DIR if os.path.exists(OUTPUT_BASE_DIR) else '.'
    return os.path.join(output_base_dir, 'interventions_{}_{}'.format(pathogen_name, p))

results_pickle = os.path.join(OUTPUT_BASE_DIR, 'all_results.pkl')
run_analysis = not os.path.exists(results_pickle)

def runner(r,
        n_visitors = bau_n_visitors,
        n_visitors_to_infect = 28,
        n_staff_to_infect = 0,
        boxes_to_drop = None,
        ban_eating_in_foyer = False,
        extract_boxes = False,
        extract_concentrations = False,
        extract_expected_infections = True,
        jp_seats = 120):

    assert n_visitors_to_infect <= n_visitors

    workplace.reset(full=True)

    # add some visitors
    for visitor in generate_visitors_for_period(
            n_visitors,
            box_enum_to_box_map,
            generators,
            remaining_seats = jp_seats,
            boxes_to_drop = boxes_to_drop,
            ban_eating_in_foyer = ban_eating_in_foyer):
        workplace.add_visitor(visitor)

    # infect some people
    workplace.infect_random_visitors(pathogen, time=-4, n=n_visitors_to_infect)
    workplace.infect_random_persons(pathogen, time=-4, n=n_staff_to_infect)

    # and run a day
    workplace.run_period(0, pathogen, emissions_calculator)
    
    # extract and return results
    res = extract_plot_data(
        workplace = workplace,
        human_box_types = HUMAN_BOX_TYPES,
        extract_boxes = extract_boxes,
        extract_concentrations = extract_concentrations,
        extract_expected_infections = extract_expected_infections)
    
    res.update({
        'n_infected': workplace.count_infected(include_visitors = True)
    })
    
    return res

## Basic Setup

In [ ]:
workplace, box_enum_to_box_map, _, _, _ = create_workplace()
pathogen = create_pathogen(pathogen_name)
emissions_calculator = create_emissions_calculator(pathogen)
generators = [VisitGenerator()]

In [ ]:
import matplotlib.font_manager as font_manager

# Add every font at the specified location
font_dir = ['../fnt']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Palatino'

## Summarise a Single Day

In [ ]:
plot_data = runner(None, n_staff_to_infect = 1)
shedding_plot(
    workplace,
    opath('shedding.pdf'),
    human_box_types = HUMAN_BOX_TYPES,
    panel_height = 3.7,
    show_expected_infections = True)

## Run Lots of Days

In [ ]:
R = 10000
process_seeder = ProcessSeeder()

def run_job(workplace, runner, R, scenario, post_reset=None, **kwargs):
    if run_analysis:
        process_seeder.reset()
        workplace.reset(full=True)
        if post_reset is not None:
            post_reset(workplace)
        with Pool(POOL_NCORES, initializer=process_seeder) as pool:
            work = pool.imap_unordered(runner, range(R))
            return [o for o in tqdm(work, total=R)], scenario

### BAU

In [ ]:
outputs_bau = run_job(workplace, runner, R, 'BAU')

### Close Mini-Q (Gallery 6)

In [ ]:
def drop_G6_runner(i):
    return runner(i, boxes_to_drop = [BOX.G6])
outputs_drop_G6 = run_job(workplace, drop_G6_runner, R, "Close Mini Q")

### Ban Eating in Foyer

In [ ]:
def ban_eating_in_foyer_runner(i):
    return runner(i, ban_eating_in_foyer = True)
outputs_ban_eating_in_foyer = run_job(workplace, ban_eating_in_foyer_runner, R, "Ban Eating Indoors")

### Ban Eating in Foyer and Close Mini-Q

In [ ]:
def ban_eating_and_G6_runner(i):
    return runner(i,
        ban_eating_in_foyer = True,
        boxes_to_drop = [BOX.G6])
outputs_ban_eating_and_G6 = run_job(
    workplace,
    ban_eating_and_G6_runner,
    R,
    "Ban Eating Indoors and Close Mini Q")

### Reduced Visitor Numbers

In [ ]:
def limit_visitors_runner(i):
    return runner(i,
        n_visitors = limited_n_visitors,
        jp_seats = 77)
outputs_reduced_visitors = run_job(
    workplace,
    limit_visitors_runner,
    R, "{} Visitors per Day".format(limited_n_visitors))

### Reflect the Existing HVAC Filters

In [ ]:
workplace, box_enum_to_box_map, _, _, _ = create_workplace(hvac_return_filtering_efficiency = .85)
outputs_hvac_filter = run_job(workplace, runner, R, "Existing HVAC Filters")

### Reflect the Existing HVAC Filters,  Masks and Ban Eating in Foyer

In [ ]:
Person.ingestion_filter_efficiency = .5
Person.shedding_filter_efficiency = .5
outputs_hvac_filter_and_masks = run_job(
    workplace,
    ban_eating_in_foyer_runner,
    R,
    "Existing HVAC Filters, Ban Eating Indoors, and Masks")

### Reflect the Existing HVAC Filters, Masks, Ban Eating in Foyer and Limited Visitors

In [ ]:
def ban_eating_in_foyer_and_limit_visitors_runner(i):
    return runner(i,
        ban_eating_in_foyer = True,
        n_visitors = limited_n_visitors,
        jp_seats = 77)
outputs_hvac_filter_masks_limit_visitors_ban_eating = run_job(
    workplace,
    ban_eating_in_foyer_and_limit_visitors_runner,
    R,
    "Existing HVAC Filters, Masks, Ban Eating Indoors, and Reduced Visitors")

### Infect One Worker

In [ ]:
workplace, box_enum_to_box_map, _, _, _ = create_workplace()
Person.ingestion_filter_efficiency = 0.
Person.shedding_filter_efficiency = 0.
def infect_a_worker_runner(i):
    return runner(i, n_visitors_to_infect = 27, n_staff_to_infect = 1)
outputs_infect_a_worker = run_job(workplace, infect_a_worker_runner, R, "Infected Staff Member")

## Combine Results

In [ ]:
if run_analysis:        
    all_results = (
        outputs_bau,
        outputs_drop_G6,
        outputs_ban_eating_in_foyer,
        outputs_ban_eating_and_G6,
        outputs_reduced_visitors,
        outputs_hvac_filter,
        outputs_hvac_filter_and_masks,
        outputs_hvac_filter_masks_limit_visitors_ban_eating,
        outputs_infect_a_worker)
    
    with open(results_pickle, 'wb') as pkl:
        pickle.dump((all_results, R), pkl)
        
else:
    with open(results_pickle, 'rb') as pkl:
        all_results, R = pickle.load(pkl)

## Plots and Tables

### Histograms

Unused because the infection numbers are so low, they did not help distinguish between the scenarios.

In [ ]:
def plot_histograms_for_output(ax, outputs):
    n_infected = np.array([o['n_infected'] for o in outputs])
    ax.set_title('Histogram of Number of Infections')
    _ = ax.hist(
        n_infected,
        bins = range(max(n_infected)+2),
        align = 'left')
    ax.set_xticks(range(max(n_infected)+1))
    plt.savefig(opath('histograms_of_number_of_infections.pdf'), bbox_inches='tight')
    return n_infected

### Boxplots

In [ ]:
def plot_boxplots_for_output(ax, outputs, ymax = None,
        show_x_labels = True,
        show_y_labels = True):
    outputs, name = outputs
    box_risks = np.array([o['box_risks'] for o in outputs])
    box_names = outputs[0]['box_names']
    ax.set_title(name)
    if show_y_labels:
        ax.set(ylabel='Number of Infections')
    if ymax is not None:
        ax.set_ylim((-.02, ymax))
    if show_x_labels:
        bpd = ax.boxplot(box_risks, labels = box_names)
        ax.tick_params(axis='x', rotation=60)
    else:
        bpd = ax.boxplot(box_risks)
        ax.get_xaxis().set_visible(False)

def plot_boxplots(all_outputs):
    ymax = max(max(o['box_risks']) for outputs in all_outputs for o in outputs[0]) * 1.05
    n_plots = len(all_outputs)
    fig, axs = plt.subplots(n_plots, figsize=(9., n_plots * 9./10*1.75))
    fig.tight_layout()
    for i, (ax, outputs) in enumerate(zip(axs, all_outputs)):
        plot_boxplots_for_output(ax, outputs, ymax,
        show_x_labels = i == n_plots-1,
        show_y_labels = i == 0)
    plt.savefig(opath('boxplots_expected_infections.pdf'), bbox_inches='tight')
    plt.show()

In [ ]:
plot_boxplots(all_results)

### Results Table

In [ ]:
aves = [(outputs[1], np.mean([o['n_infected'] for o in outputs[0]])) for outputs in all_results]
means = pd.DataFrame({'scenario': [a[0] for a in aves], 'mean': [a[1] for a in aves]})
with pd.option_context("max_colwidth", 1000):
    means_latex = means.to_latex(
        header=['Scenario', 'Average Number Infected'],
        index = False,
        float_format='%0.2f',
        caption=r'Average number of infections under the BAU and for each intervention.',
        label='tab:questacon:interventions_mean_number_of_infections',
        position='htbp')
with open(opath('means.tex'), 'w') as outf: outf.write(means_latex)
print(means)